# NHL Player Preprocessing

This file is used to preprocess the NHL player data.

- Exclude Rookies

    - Only include players who surpass rookie eligibility (Calder Memorial Trophy) limits:

        - played $\geq 25$ NHL games in a single season, or

        - In modern practice, only NHL experience is practically considered.

            - International pro league (KHL, SHL, Liiga, etc.) do *not disqualify* players from Calder Trophy eligibility.

        - players must be 26 years old or younger by September 15th of the season in question.

- Exclude Fringe Players

    - Min 2 seasons in NHL.

    - At least 41 `gp_regular_max_per_season` in one regular season.

- Divide players by position (player_pos_official) -> (FW, D)


## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os

## Read in Data

In [2]:
# Load players' metadata
nhl_players_metadata = pd.read_csv('../dataset/nhl_players_metadata_facts_merged_final.csv')
nhl_players_stats = pd.read_csv('../dataset/nhl_players_official_stats_with_id_sorted.csv')

In [3]:
nhl_players_metadata.shape

(3928, 18)

In [4]:
nhl_players_stats.shape

(106409, 37)

## Preprocess `nhl_players_stats`'s data types

## NHL Stats Aggregation

- This step will aggregate the stats by player and league.

    - We will Calculate their NHL career stats and `min_gp_regular`

    - We will also calculate their career stats in other leagues (KHL, SHL, Liiga, etc.)

In [5]:
# Copy only NHL stats
nhl_career_stats = nhl_players_stats[nhl_players_stats['league'] == 'NHL'].copy()

# Get the first NHL season year for calculating the rookie ages later.
nhl_career_stats['first_season'] = nhl_career_stats.groupby('player_id')['season'].transform('min')
nhl_career_stats

,player_id,player_name_official,season,league,team,gp_regular,g_regular,a_regular,p_regular,plus_minus_regular,...,ppp_playoffs,shg_playoffs,shp_playoffs,toi_per_game_playoffs,gwg_playoffs,otg_playoffs,sog_playoffs,shooting_pct_playoffs,fo_pct_playoffs,first_season
6,1.0,Joe Sakic,1988-89,NHL,Quebec Nordiques,70.0,23.0,39.0,62.0,-36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-89
7,1.0,Joe Sakic,1989-90,NHL,Quebec Nordiques,80.0,39.0,63.0,102.0,-40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-89
8,1.0,Joe Sakic,1990-91,NHL,Quebec Nordiques,80.0,48.0,61.0,109.0,-26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-89
10,1.0,Joe Sakic,1991-92,NHL,Quebec Nordiques,69.0,29.0,65.0,94.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1988-89
11,1.0,Joe Sakic,1992-93,NHL,Quebec Nordiques,78.0,48.0,57.0,105.0,-3.0,...,2.0,0.0,0.0,NaN,0.0,0.0,24.0,12.5,NaN,1988-89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106262,3922.0,Calum Ritchie,2024-25,NHL,Colorado Avalanche,7.0,1.0,0.0,1.0,-7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25
106313,3925.0,Daniil Misyul,2024-25,NHL,New Jersey Devils,1.0,0.0,0.0,0.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25
106336,3926.0,Nate Clurman,2024-25,NHL,Pittsburgh Penguins,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25
106355,3927.0,Jacob Gaucher,2024-25,NHL,Philadelphia Flyers,4.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-25


In [6]:
# Convert columns that should be numeric
numeric_cols = [
    "gp_regular", "g_regular", "a_regular", "p_regular",
    "plus_minus_regular", "pim_regular", "ppg_regular", "ppp_regular",
    "shg_regular", "shp_regular", "gwg_regular", "otg_regular", "sog_regular",
    "shooting_pct_regular", "fo_pct_regular",
    "gp_playoffs", "g_playoffs", "a_playoffs", "p_playoffs",
    "plus_minus_playoffs", "pim_playoffs", "ppg_playoffs", "ppp_playoffs",
    "shg_playoffs", "shp_playoffs", "gwg_playoffs", "otg_playoffs", "sog_playoffs",
    "shooting_pct_playoffs", "fo_pct_playoffs"
]

for col in numeric_cols:
    nhl_career_stats[col] = pd.to_numeric(nhl_career_stats[col], errors="coerce")

# Handle TOI (time on ice) separately if it's "MM:SS" or "HH:MM:SS"
def convert_toi_to_minutes(toi_str):
    if pd.isna(toi_str): return None
    parts = str(toi_str).split(":")
    if len(parts) == 2:  # MM:SS
        m, s = map(int, parts)
        return m + s/60
    elif len(parts) == 3:  # HH:MM:SS
        h, m, s = map(int, parts)
        return h*60 + m + s/60
    return None

nhl_career_stats["toi_per_game_regular"] = nhl_career_stats["toi_per_game_regular"].apply(convert_toi_to_minutes)
nhl_career_stats["toi_per_game_playoffs"] = nhl_career_stats["toi_per_game_playoffs"].apply(convert_toi_to_minutes)

# Add extra stats before aggregation
agg_funcs = {
    "gp_regular": ["sum", "min", "max"],  # total GP, min GP, max GP in a season
    "g_regular": "sum",
    "a_regular": "sum",
    "p_regular": "sum",
    "plus_minus_regular": "sum",
    "pim_regular": "sum",
    "ppg_regular": "sum",
    "ppp_regular": "sum",
    "shg_regular": "sum",
    "shp_regular": "sum",
    "gwg_regular": "sum",
    "otg_regular": "sum",
    "sog_regular": "sum",
    "shooting_pct_regular": "mean",  # percentage -> average
    "fo_pct_regular": "mean",        # percentage -> average
    "toi_per_game_regular": "mean",  # average TOI per game
    "gp_playoffs": "sum",
    "g_playoffs": "sum",
    "a_playoffs": "sum",
    "p_playoffs": "sum",
    "plus_minus_playoffs": "sum",
    "pim_playoffs": "sum",
    "ppg_playoffs": "sum",
    "ppp_playoffs": "sum",
    "shg_playoffs": "sum",
    "shp_playoffs": "sum",
    "gwg_playoffs": "sum",
    "otg_playoffs": "sum",
    "sog_playoffs": "sum",
    "shooting_pct_playoffs": "mean",  # %
    "fo_pct_playoffs": "mean",        # %
    "toi_per_game_playoffs": "mean",
    "season": "nunique"  # number of distinct NHL seasons
}

# Aggregate
nhl_career_stats = nhl_career_stats.groupby(
    ["player_id", "player_name_official", "first_season"]
).agg(agg_funcs)

# Flatten MultiIndex column names
nhl_career_stats.columns = [
    "_".join(col).strip("_") for col in nhl_career_stats.columns.values
]

# Rename for clarity
nhl_career_stats = nhl_career_stats.rename(
    columns={
        "gp_regular_sum": "gp_regular_total",
        "gp_regular_min": "gp_regular_min_per_season",
        "gp_regular_max": "gp_regular_max_per_season",
        "season_nunique": "seasons_played"
    }
).reset_index()

# Add seasons_with_gp_over_6 (custom calculation)
seasons_over_6 = (
    nhl_players_stats[nhl_players_stats["league"] == "NHL"]
    .groupby(["player_id", "season"])["gp_regular"]
    .sum()
    .reset_index()
)

seasons_over_6["over_6"] = seasons_over_6["gp_regular"] > 6

seasons_with_gp_over_6 = (
    seasons_over_6.groupby("player_id")["over_6"].sum().reset_index()
    .rename(columns={"over_6": "seasons_with_gp_over_6"})
)

# Merge back into career stats
nhl_career_stats = nhl_career_stats.merge(
    seasons_with_gp_over_6, on="player_id", how="left"
)

In [7]:
# --- Add per-game stats (regular season) ---
nhl_career_stats["g_regular_per_game"] = nhl_career_stats["g_regular_sum"] / nhl_career_stats["gp_regular_total"]
nhl_career_stats["a_regular_per_game"] = nhl_career_stats["a_regular_sum"] / nhl_career_stats["gp_regular_total"]
nhl_career_stats["p_regular_per_game"] = nhl_career_stats["p_regular_sum"] / nhl_career_stats["gp_regular_total"]
nhl_career_stats["sog_regular_per_game"] = nhl_career_stats["sog_regular_sum"] / nhl_career_stats["gp_regular_total"]
nhl_career_stats["pim_regular_per_game"] = nhl_career_stats["pim_regular_sum"] / nhl_career_stats["gp_regular_total"]

# --- Per-game stats (playoffs) ---
nhl_career_stats["g_playoffs_per_game"] = nhl_career_stats["g_playoffs_sum"] / nhl_career_stats["gp_playoffs_sum"]
nhl_career_stats["a_playoffs_per_game"] = nhl_career_stats["a_playoffs_sum"] / nhl_career_stats["gp_playoffs_sum"]
nhl_career_stats["p_playoffs_per_game"] = nhl_career_stats["p_playoffs_sum"] / nhl_career_stats["gp_playoffs_sum"]
nhl_career_stats["sog_playoffs_per_game"] = nhl_career_stats["sog_playoffs_sum"] / nhl_career_stats["gp_playoffs_sum"]
nhl_career_stats["pim_playoffs_per_game"] = nhl_career_stats["pim_playoffs_sum"] / nhl_career_stats["gp_playoffs_sum"]

# --- Add per-season stats (regular season) ---
nhl_career_stats["gp_regular_per_season"] = nhl_career_stats["gp_regular_total"] / nhl_career_stats["seasons_played"]
nhl_career_stats["g_regular_per_season"] = nhl_career_stats["g_regular_sum"] / nhl_career_stats["seasons_played"]
nhl_career_stats["a_regular_per_season"] = nhl_career_stats["a_regular_sum"] / nhl_career_stats["seasons_played"]
nhl_career_stats["p_regular_per_season"] = nhl_career_stats["p_regular_sum"] / nhl_career_stats["seasons_played"]

# Handle divide-by-zero (e.g., if no games played or no playoff games)
nhl_career_stats = nhl_career_stats.fillna(0)


In [8]:
regular_cols = [
    "gp_regular_total", "gp_regular_min_per_season", "gp_regular_max_per_season",
    "g_regular_sum", "a_regular_sum", "p_regular_sum", "plus_minus_regular_sum", "pim_regular_sum",
    "ppg_regular_sum", "ppp_regular_sum", "shg_regular_sum", "shp_regular_sum",
    "gwg_regular_sum", "otg_regular_sum", "sog_regular_sum",
    "shooting_pct_regular_mean", "fo_pct_regular_mean", "toi_per_game_regular_mean",
    # per-game
    "g_regular_per_game", "a_regular_per_game", "p_regular_per_game",
    "sog_regular_per_game", "pim_regular_per_game",
    # per-season
    "gp_regular_per_season", "g_regular_per_season",
    "a_regular_per_season", "p_regular_per_season"
]

playoff_cols = [
    "gp_playoffs_sum",
    "g_playoffs_sum", "a_playoffs_sum", "p_playoffs_sum", "plus_minus_playoffs_sum", "pim_playoffs_sum",
    "ppg_playoffs_sum", "ppp_playoffs_sum", "shg_playoffs_sum", "shp_playoffs_sum",
    "gwg_playoffs_sum", "otg_playoffs_sum", "sog_playoffs_sum",
    "shooting_pct_playoffs_mean", "fo_pct_playoffs_mean", "toi_per_game_playoffs_mean",
    # per-game
    "g_playoffs_per_game", "a_playoffs_per_game", "p_playoffs_per_game",
    "sog_playoffs_per_game", "pim_playoffs_per_game",
]

meta_cols = [
    "first_season", "seasons_played", "seasons_with_gp_over_6"
]

# Now build the full order
ordered_cols = ["player_id", "player_name_official"] + meta_cols + regular_cols + playoff_cols

# In case there are any extras we missed, append them at the end
extras = [c for c in nhl_career_stats.columns if c not in ordered_cols]
ordered_cols += extras

# Reorder DataFrame
nhl_career_stats = nhl_career_stats[ordered_cols]


In [9]:
nhl_career_stats

,player_id,player_name_official,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,g_regular_sum,a_regular_sum,...,otg_playoffs_sum,sog_playoffs_sum,shooting_pct_playoffs_mean,fo_pct_playoffs_mean,toi_per_game_playoffs_mean,g_playoffs_per_game,a_playoffs_per_game,p_playoffs_per_game,sog_playoffs_per_game,pim_playoffs_per_game
0,1.0,Joe Sakic,1988-89,20,20,1378.0,15.0,84.0,625.0,1016.0,...,8.0,582.0,14.923077,53.700000,350.955556,0.488372,0.604651,1.093023,3.383721,0.453488
1,2.0,Peter Forsberg,1994-95,13,12,708.0,2.0,82.0,249.0,636.0,...,1.0,318.0,19.291667,50.233333,178.305556,0.419847,0.679389,1.099237,2.427481,1.091603
2,3.0,Milan Hejduk,1998-99,14,14,1020.0,29.0,82.0,375.0,430.0,...,2.0,281.0,13.600000,39.911111,18.679630,0.303571,0.375000,0.678571,2.508929,0.250000
3,4.0,Alex Tanguay,1999-00,16,16,1088.0,16.0,82.0,283.0,580.0,...,1.0,125.0,13.444444,20.066667,16.933333,0.193878,0.408163,0.602041,1.275510,0.428571
4,5.0,Chris Drury,1998-99,12,12,892.0,24.0,82.0,255.0,360.0,...,4.0,314.0,14.055556,52.244444,16.529630,0.348148,0.311111,0.659259,2.325926,0.340741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3921,3922.0,Calum Ritchie,2024-25,1,1,7.0,7.0,7.0,1.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3922,3925.0,Daniil Misyul,2024-25,1,0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3923,3926.0,Nate Clurman,2024-25,1,0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3924,3927.0,Jacob Gaucher,2024-25,1,0,4.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
# Write to CSV
nhl_career_stats.to_csv('../dataset/nhl_players_career_stats.csv', index=False, encoding='utf-8-sig')

## Exclude Rookies
- Only include players who surpass rookie eligibility (Calder Memorial Trophy) limits:

    1. players must be 26 years old or younger by September 15th of the season in question.

    2. played $\geq 25$ NHL games in a single season, or

    3. played $ \leq 6$ NHL games in any two previous seasons.

    - In modern practice, only NHL experience is practically considered.

        - International pro league (KHL, SHL, Liiga, etc.) do *not disqualify* players from Calder Trophy eligibility.


In [11]:
# Load Metadata
nhl_players_metadata = pd.read_csv('../dataset/nhl_players_metadata_facts_merged_final.csv')

In [12]:
# Load Career Stats
nhl_career_stats = pd.read_csv('../dataset/nhl_players_career_stats.csv')

In [13]:
# Parse the age_by_2025 using date_of_birth (MM/DD/YYYY)
target_date = pd.Timestamp("2025-09-15")
nhl_players_metadata['age_by_2025'] = pd.to_datetime(
    nhl_players_metadata['date_of_birth'], format='%m/%d/%Y'
).apply(lambda dob: int((target_date - dob).days // 365))

In [14]:
# Merge Career Stats and Metadata
nhl_players = nhl_career_stats.merge(nhl_players_metadata, on=['player_id', 'player_name_official'], how='left')
nhl_players

,player_id,player_name_official,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,g_regular_sum,a_regular_sum,...,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description,age_by_2025
0,1.0,Joe Sakic,1988-89,20,20,1378.0,15.0,84.0,625.0,1016.0,...,Canada,180,88,L,NaN,NaN,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...,56
1,2.0,Peter Forsberg,1994-95,13,12,708.0,2.0,82.0,249.0,636.0,...,Sweden,185,93,L,"['Cerebral Tactician', 'Playmaker', 'Two-Way C...",NaN,"('1', '6', '1991')","['1-time TV-Pucken Gold Medal', '1-time J18 SM...",Arguably the greatest Swedish forward to ever ...,52
2,3.0,Milan Hejduk,1998-99,14,14,1020.0,29.0,82.0,375.0,430.0,...,Czechia / USA,183,87,R,['Sniper'],NaN,"('4', '87', '1994')","['1-time Olympic Gold Medal', '1-time World Ch...",Father: Milan Hejduk\r\nCousin: Tomas Hejduk\r...,49
3,4.0,Alex Tanguay,1999-00,16,16,1088.0,16.0,82.0,283.0,580.0,...,Canada,185,88,L,['Playmaker'],NaN,"('1', '12', '1998')","['1-time CHL All-Rookie Team', '1-time NHL Sta...",Brother: Maxime Tanguay\r\nCousin: Tyler Tangu...,45
4,5.0,Chris Drury,1998-99,12,12,892.0,24.0,82.0,255.0,360.0,...,USA,178,86,R,NaN,NaN,"('3', '72', '1994')","['2-time Olympic Silver Medal', '1-time World ...",Son: Luke Drury\r\nBrothers: Ted Drury & Jim D...,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3921,3922.0,Calum Ritchie,2024-25,1,1,7.0,7.0,7.0,1.0,0.0,...,Canada,188,86,R,"['Playmaker', 'Puckhandler', 'Two-Way Center']",New York Islanders / Signed,"('1', '27', '2023')","['1-time Hlinka Gretzky Cup Gold Medal', '1-ti...",Ritchie’s hockey sense is his defining quality...,20
3922,3925.0,Daniil Misyul,2024-25,1,0,1.0,1.0,1.0,0.0,0.0,...,Russia / Belarus,191,89,L,NaN,NaN,"('3', '70', '2019')","['1-time U20 WJC Silver Medal', '1-time U17 WH...",Brother: Oleg Misyul,24
3923,3926.0,Nate Clurman,2024-25,1,0,1.0,1.0,1.0,0.0,0.0,...,USA,188,93,R,['Defensive Defenseman'],Montréal Canadiens / Signed,"('6', '161', '2016')",['1-time NCAA (B1G) Champion'],NaN,27
3924,3927.0,Jacob Gaucher,2024-25,1,0,4.0,4.0,4.0,0.0,0.0,...,Canada,191,84,R,NaN,Philadelphia Flyers / Signed,NaN,[],Father: Yannick Gaucher Brother: Nathan Gaucher,24


In [15]:
# Parse the first_season and calculate the rookie_age by rookie_age = (first_season_year/09/15 - date_of_birth)
nhl_players['first_season_year'] = nhl_players['first_season'].apply(lambda x: int(x.split('-')[0]))
nhl_players['rookie_age'] = (pd.to_datetime(nhl_players['first_season_year'], format='%Y') - pd.to_datetime(nhl_players['date_of_birth'], format='%m/%d/%Y')).apply(lambda x: x.days // 365)

# Drop first_season_year
nhl_players = nhl_players.drop(['first_season_year'], axis=1)

In [16]:
# Reorder Columns
nhl_players = nhl_players[['player_id', 'player_name_official', 'rookie_age', 'age_by_2025'] + [c for c in nhl_players.columns if c not in ['player_id', 'player_name_official', 'rookie_age', 'age_by_2025']]]

In [17]:
nhl_players

,player_id,player_name_official,rookie_age,age_by_2025,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,...,date_of_birth,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,1.0,Joe Sakic,18,56,1988-89,20,20,1378.0,15.0,84.0,...,7/7/1969,Canada,180,88,L,NaN,NaN,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...
1,2.0,Peter Forsberg,20,52,1994-95,13,12,708.0,2.0,82.0,...,7/20/1973,Sweden,185,93,L,"['Cerebral Tactician', 'Playmaker', 'Two-Way C...",NaN,"('1', '6', '1991')","['1-time TV-Pucken Gold Medal', '1-time J18 SM...",Arguably the greatest Swedish forward to ever ...
2,3.0,Milan Hejduk,21,49,1998-99,14,14,1020.0,29.0,82.0,...,2/14/1976,Czechia / USA,183,87,R,['Sniper'],NaN,"('4', '87', '1994')","['1-time Olympic Gold Medal', '1-time World Ch...",Father: Milan Hejduk\r\nCousin: Tomas Hejduk\r...
3,4.0,Alex Tanguay,19,45,1999-00,16,16,1088.0,16.0,82.0,...,11/21/1979,Canada,185,88,L,['Playmaker'],NaN,"('1', '12', '1998')","['1-time CHL All-Rookie Team', '1-time NHL Sta...",Brother: Maxime Tanguay\r\nCousin: Tyler Tangu...
4,5.0,Chris Drury,21,49,1998-99,12,12,892.0,24.0,82.0,...,8/20/1976,USA,178,86,R,NaN,NaN,"('3', '72', '1994')","['2-time Olympic Silver Medal', '1-time World ...",Son: Luke Drury\r\nBrothers: Ted Drury & Jim D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3921,3922.0,Calum Ritchie,18,20,2024-25,1,1,7.0,7.0,7.0,...,1/21/2005,Canada,188,86,R,"['Playmaker', 'Puckhandler', 'Two-Way Center']",New York Islanders / Signed,"('1', '27', '2023')","['1-time Hlinka Gretzky Cup Gold Medal', '1-ti...",Ritchie’s hockey sense is his defining quality...
3922,3925.0,Daniil Misyul,23,24,2024-25,1,0,1.0,1.0,1.0,...,10/20/2000,Russia / Belarus,191,89,L,NaN,NaN,"('3', '70', '2019')","['1-time U20 WJC Silver Medal', '1-time U17 WH...",Brother: Oleg Misyul
3923,3926.0,Nate Clurman,25,27,2024-25,1,0,1.0,1.0,1.0,...,5/8/1998,USA,188,93,R,['Defensive Defenseman'],Montréal Canadiens / Signed,"('6', '161', '2016')",['1-time NCAA (B1G) Champion'],NaN
3924,3927.0,Jacob Gaucher,22,24,2024-25,1,0,4.0,4.0,4.0,...,3/9/2001,Canada,191,84,R,NaN,Philadelphia Flyers / Signed,NaN,[],Father: Yannick Gaucher Brother: Nathan Gaucher


In [18]:
# Write to CSV
nhl_players.to_csv('../dataset/preprocess/nhl_players_career_stats_with_metadata.csv', index=False, encoding='utf-8-sig')

In [19]:
# Condition 1: Age eligibility
cond_age = nhl_players['age_by_2025'] <= 26

# Condition 2: Must NOT have played >= 25 games in any prior season
cond_single_season = nhl_players['gp_regular_max_per_season'] < 25

# Condition 3: Must NOT have played > 6 games in two or more previous seasons
cond_two_seasons = nhl_players['seasons_with_gp_over_6'] < 2

# Condition 4: young stars rule - seasons_played <= 3 & age < 24 (Forwards) or 26 for (Defensemen)
cond_defensemen = (nhl_players['player_pos_official'] == 'D') & (nhl_players['age_by_2025'] < 26) & (nhl_players['seasons_played'] <= 3)
cond_forwards = (nhl_players['player_pos_official'] != 'D') & (nhl_players['age_by_2025'] < 24) & (nhl_players['seasons_played'] <= 3)

# Combine conditions
rookies_players_id = nhl_players[
    (cond_age & cond_single_season & cond_two_seasons) | (cond_defensemen | cond_forwards)
]['player_id']

rookies_players_id

272      273.0
280      281.0
281      282.0
285      286.0
303      304.0
         ...  
3918    3919.0
3919    3920.0
3921    3922.0
3922    3925.0
3924    3927.0
Name: player_id, Length: 283, dtype: float64

In [20]:
# Show Rookies
rookies = nhl_players[
    nhl_players['player_id'].isin(rookies_players_id)
]

rookies

,player_id,player_name_official,rookie_age,age_by_2025,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,...,date_of_birth,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
272,273.0,Sampo Ranta,20,25,2021-22,2,1,16.0,6.0,10.0,...,5/31/2000,Finland,189,88,L,"['Dangler', 'Offensive Forward', 'Speedster']",Colorado Avalanche / Unsigned,"('3', '78', '2018')","['1-time U18 WJC Gold Medal', '1-time NCAA (B1...",NaN
280,281.0,Shane Bowers,22,26,2022-23,3,1,13.0,1.0,8.0,...,7/30/1999,Canada,189,84,L,NaN,San Jose Sharks / Signed,"('1', '28', '2017')","['1-time NCAA (Hockey East) Champion', '1-time...",NaN
281,282.0,Oskar Olausson,19,22,2022-23,3,0,4.0,1.0,2.0,...,11/10/2002,Sweden,187,82,L,NaN,San Jose Sharks / Signed,"('1', '28', '2021')","['1-time Hlinka Gretzky Cup Bronze Medal', '1-...",Olausson’s skating posture allows him to effor...
285,286.0,Jean-Luc Foudy,19,23,2022-23,2,1,13.0,4.0,9.0,...,5/13/2002,Canada,180,80,R,['Speedster'],NaN,"('3', '75', '2020')",['1-time Hlinka Gretzky Cup Silver Medal'],It's not often that the clear-cut best skater ...
303,304.0,Jason Polin,23,26,2023-24,2,1,9.0,2.0,7.0,...,6/17/1999,USA,183,90,R,NaN,Colorado Avalanche / Signed,NaN,['1-time NCAA (West) Second All-American Team'],Brother: Ryan Polin\r\nSister: Jennifer Polin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3918,3919.0,Cameron Crotty,23,26,2023-24,2,0,2.0,1.0,1.0,...,5/5/1999,Canada,191,96,R,['Defensive Defenseman'],NaN,"('3', '82', '2017')","['1-time CCHL Top Prospect Award', '1-time NCA...",NaN
3919,3920.0,Maksymilian Szuber,20,23,2023-24,1,0,1.0,1.0,1.0,...,8/25/2002,Germany / Poland,191,91,L,['Defensive Defenseman'],Utah Mammoth / Signed,"('6', '163', '2022')","['1-time World Championship Silver Medal', '1-...",NaN
3921,3922.0,Calum Ritchie,18,20,2024-25,1,1,7.0,7.0,7.0,...,1/21/2005,Canada,188,86,R,"['Playmaker', 'Puckhandler', 'Two-Way Center']",New York Islanders / Signed,"('1', '27', '2023')","['1-time Hlinka Gretzky Cup Gold Medal', '1-ti...",Ritchie’s hockey sense is his defining quality...
3922,3925.0,Daniil Misyul,23,24,2024-25,1,0,1.0,1.0,1.0,...,10/20/2000,Russia / Belarus,191,89,L,NaN,NaN,"('3', '70', '2019')","['1-time U20 WJC Silver Medal', '1-time U17 WH...",Brother: Oleg Misyul


In [21]:
# Write rookies to CSV
rookies.to_csv('../dataset/preprocess/nhl_rookies.csv', index=False, encoding='utf-8-sig')

In [22]:
# Remove rookies from nhl_players
nhl_players_without_rookies = nhl_players[~nhl_players['player_id'].isin(rookies_players_id)]
nhl_players_without_rookies

,player_id,player_name_official,rookie_age,age_by_2025,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,...,date_of_birth,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,1.0,Joe Sakic,18,56,1988-89,20,20,1378.0,15.0,84.0,...,7/7/1969,Canada,180,88,L,NaN,NaN,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...
1,2.0,Peter Forsberg,20,52,1994-95,13,12,708.0,2.0,82.0,...,7/20/1973,Sweden,185,93,L,"['Cerebral Tactician', 'Playmaker', 'Two-Way C...",NaN,"('1', '6', '1991')","['1-time TV-Pucken Gold Medal', '1-time J18 SM...",Arguably the greatest Swedish forward to ever ...
2,3.0,Milan Hejduk,21,49,1998-99,14,14,1020.0,29.0,82.0,...,2/14/1976,Czechia / USA,183,87,R,['Sniper'],NaN,"('4', '87', '1994')","['1-time Olympic Gold Medal', '1-time World Ch...",Father: Milan Hejduk\r\nCousin: Tomas Hejduk\r...
3,4.0,Alex Tanguay,19,45,1999-00,16,16,1088.0,16.0,82.0,...,11/21/1979,Canada,185,88,L,['Playmaker'],NaN,"('1', '12', '1998')","['1-time CHL All-Rookie Team', '1-time NHL Sta...",Brother: Maxime Tanguay\r\nCousin: Tyler Tangu...
4,5.0,Chris Drury,21,49,1998-99,12,12,892.0,24.0,82.0,...,8/20/1976,USA,178,86,R,NaN,NaN,"('3', '72', '1994')","['2-time Olympic Silver Medal', '1-time World ...",Son: Luke Drury\r\nBrothers: Ted Drury & Jim D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3915,3916.0,Milos Kelemen,22,26,2022-23,2,2,24.0,10.0,14.0,...,7/6/1999,Slovakia,188,99,L,NaN,NaN,NaN,"['1-time Olympic Bronze Medal', '1-time Czech ...",NaN
3917,3918.0,Nikolai Kovalenko,24,25,2024-25,1,1,57.0,28.0,29.0,...,10/17/1999,Russia / USA,179,82,L,NaN,NaN,"('6', '171', '2018')","['1-time U17 WHC Silver Medal', '1-time WJAC-1...",Father: Andrei Kovalenko\r\nBrothers: Danila K...
3920,3921.0,Patrik Koch,26,28,2023-24,1,0,1.0,1.0,1.0,...,12/8/1996,Slovakia,186,86,L,"['Defensive Defenseman', 'Physical Defenseman']",NaN,NaN,[],NaN
3923,3926.0,Nate Clurman,25,27,2024-25,1,0,1.0,1.0,1.0,...,5/8/1998,USA,188,93,R,['Defensive Defenseman'],Montréal Canadiens / Signed,"('6', '161', '2016')",['1-time NCAA (B1G) Champion'],NaN


In [23]:
# Write to CSV
nhl_players_without_rookies.to_csv('../dataset/preprocess/nhl_players_career_stats_with_metadata_without_rookies.csv', index=False, encoding='utf-8-sig')

## Exclude Fringe Players

- Fringe Players are defined as:

    1. players who have played less than 2 seasons in the NHL, or

    2. players who have not played in at least 41 games in a single season.

In [24]:
# Load NHL Players without Rookies
nhl_players_without_rookies = pd.read_csv('../dataset/preprocess/nhl_players_career_stats_with_metadata_without_rookies.csv')

In [25]:
nhl_players_without_rookies.shape

(3643, 71)

In [26]:
# Condition 1: played less than 2 seasons
cond_seasons = nhl_players_without_rookies['seasons_played'] < 2

# Condition 2: never played ≥ 41 games in a single season
cond_gp = nhl_players_without_rookies['gp_regular_max_per_season'] < 41

# Combine conditions
fringe_players_id = nhl_players_without_rookies[
    cond_seasons | cond_gp
]['player_id']


In [27]:
fringe_players_id

25        26.0
26        27.0
27        28.0
37        38.0
38        39.0
         ...  
3638    3916.0
3639    3918.0
3640    3921.0
3641    3926.0
3642    3928.0
Name: player_id, Length: 1267, dtype: float64

In [28]:
# Fringe players
nhl_fringe_players = nhl_players_without_rookies[
    nhl_players_without_rookies['player_id'].isin(fringe_players_id)
]

In [29]:
nhl_fringe_players

,player_id,player_name_official,rookie_age,age_by_2025,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,...,date_of_birth,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
25,26.0,Rob Shearer,23,48,2000-01,1,0,2.0,2.0,2.0,...,10/19/1976,Canada,175,85,R,NaN,NaN,NaN,"['1-time AHL Calder Cup Champion', '1-time DEL...",NaN
26,27.0,Joel Prpic,22,51,1997-98,3,1,18.0,1.0,14.0,...,9/25/1974,Canada / Croatia,200,107,L,NaN,NaN,"('9', '233', '1993')","['2-time ACH Champion', '1-time AHL Calder Cup...",Brother-in-law: Robb Tallas
27,28.0,Yuri Babenko,22,47,2000-01,1,0,3.0,3.0,3.0,...,1/2/1978,Russia,183,87,L,NaN,NaN,"('2', '51', '1996')","['1-time U18 EJC Gold Medal', '2-time KHL Gaga...",NaN
37,38.0,Kelly Fairchild,21,52,1995-96,4,2,34.0,1.0,22.0,...,4/9/1973,USA,181,90,L,NaN,NaN,"('7', '152', '1991')","['1-time Asia League Champion', '2-time DEL Ch...",NaN
38,39.0,Jordan Krestanovich,19,44,2001-02,2,2,22.0,8.0,14.0,...,6/14/1981,Canada,186,82,L,NaN,NaN,"('5', '152', '1999')",['1-time WHL Champion'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3638,3916.0,Milos Kelemen,22,26,2022-23,2,2,24.0,10.0,14.0,...,7/6/1999,Slovakia,188,99,L,NaN,NaN,NaN,"['1-time Olympic Bronze Medal', '1-time Czech ...",NaN
3639,3918.0,Nikolai Kovalenko,24,25,2024-25,1,1,57.0,28.0,29.0,...,10/17/1999,Russia / USA,179,82,L,NaN,NaN,"('6', '171', '2018')","['1-time U17 WHC Silver Medal', '1-time WJAC-1...",Father: Andrei Kovalenko\r\nBrothers: Danila K...
3640,3921.0,Patrik Koch,26,28,2023-24,1,0,1.0,1.0,1.0,...,12/8/1996,Slovakia,186,86,L,"['Defensive Defenseman', 'Physical Defenseman']",NaN,NaN,[],NaN
3641,3926.0,Nate Clurman,25,27,2024-25,1,0,1.0,1.0,1.0,...,5/8/1998,USA,188,93,R,['Defensive Defenseman'],Montréal Canadiens / Signed,"('6', '161', '2016')",['1-time NCAA (B1G) Champion'],NaN


In [30]:
# Write to CSV
nhl_fringe_players.to_csv('../dataset/preprocess/nhl_fringe_players.csv', index=False, encoding='utf-8-sig')

In [31]:
# Remove fringe players from nhl_players_without_rookies
nhl_players_without_rookies_and_fringe = nhl_players_without_rookies[~nhl_players_without_rookies['player_id'].isin(fringe_players_id)]

In [32]:
nhl_players_without_rookies_and_fringe

,player_id,player_name_official,rookie_age,age_by_2025,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,...,date_of_birth,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,1.0,Joe Sakic,18,56,1988-89,20,20,1378.0,15.0,84.0,...,7/7/1969,Canada,180,88,L,NaN,NaN,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...
1,2.0,Peter Forsberg,20,52,1994-95,13,12,708.0,2.0,82.0,...,7/20/1973,Sweden,185,93,L,"['Cerebral Tactician', 'Playmaker', 'Two-Way C...",NaN,"('1', '6', '1991')","['1-time TV-Pucken Gold Medal', '1-time J18 SM...",Arguably the greatest Swedish forward to ever ...
2,3.0,Milan Hejduk,21,49,1998-99,14,14,1020.0,29.0,82.0,...,2/14/1976,Czechia / USA,183,87,R,['Sniper'],NaN,"('4', '87', '1994')","['1-time Olympic Gold Medal', '1-time World Ch...",Father: Milan Hejduk\r\nCousin: Tomas Hejduk\r...
3,4.0,Alex Tanguay,19,45,1999-00,16,16,1088.0,16.0,82.0,...,11/21/1979,Canada,185,88,L,['Playmaker'],NaN,"('1', '12', '1998')","['1-time CHL All-Rookie Team', '1-time NHL Sta...",Brother: Maxime Tanguay\r\nCousin: Tyler Tangu...
4,5.0,Chris Drury,21,49,1998-99,12,12,892.0,24.0,82.0,...,8/20/1976,USA,178,86,R,NaN,NaN,"('3', '72', '1994')","['2-time Olympic Silver Medal', '1-time World ...",Son: Luke Drury\r\nBrothers: Ted Drury & Jim D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,3889.0,Matt Jones,21,42,2005-06,3,3,106.0,16.0,45.0,...,8/8/1983,USA,183,98,L,NaN,NaN,"('3', '80', '2002')",['1-time USHL Clark Cup Champion'],Brother: Zach Jones
3619,3896.0,Boris Valabik,20,39,2007-08,3,3,80.0,7.0,50.0,...,2/14/1986,Slovakia,201,105,L,"['Defensive Defenseman', 'Enforcer', 'Heavy Hi...",NaN,"('1', '10', '2004')","['1-time U18 WJC Silver Medal', '1-time AHL Ca...",A defenseman with a physical style. Useful in ...
3620,3897.0,Viktor Tikhonov,19,37,2008-09,2,2,111.0,11.0,61.0,...,5/12/1988,Russia,187,87,R,NaN,NaN,"('1', '28', '2008')","['1-time World Championship Gold Medal', '1-ti...",Tikhonov is a forward with fine offensive abil...
3633,3910.0,Mario Kempe,28,37,2017-18,2,2,70.0,18.0,52.0,...,9/19/1988,Sweden,183,86,L,NaN,NaN,"('5', '122', '2007')",['1-time U20 WJC Silver Medal'],Kempe is very good skater with good accelerati...


In [33]:
# Write to CSV
nhl_players_without_rookies_and_fringe.to_csv('../dataset/preprocess/nhl_players_career_stats_with_metadata_without_rookies_and_fringe.csv', index=False, encoding='utf-8-sig')

## Divde Players into Forwards and Defensemen

In [34]:
# Load NHL Players without Rookies and Fringe Players
nhl_players_without_rookies_and_fringe = pd.read_csv('../dataset/preprocess/nhl_players_career_stats_with_metadata_without_rookies_and_fringe.csv')

In [35]:
nhl_players_without_rookies_and_fringe

,player_id,player_name_official,rookie_age,age_by_2025,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,...,date_of_birth,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,1.0,Joe Sakic,18,56,1988-89,20,20,1378.0,15.0,84.0,...,7/7/1969,Canada,180,88,L,NaN,NaN,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...
1,2.0,Peter Forsberg,20,52,1994-95,13,12,708.0,2.0,82.0,...,7/20/1973,Sweden,185,93,L,"['Cerebral Tactician', 'Playmaker', 'Two-Way C...",NaN,"('1', '6', '1991')","['1-time TV-Pucken Gold Medal', '1-time J18 SM...",Arguably the greatest Swedish forward to ever ...
2,3.0,Milan Hejduk,21,49,1998-99,14,14,1020.0,29.0,82.0,...,2/14/1976,Czechia / USA,183,87,R,['Sniper'],NaN,"('4', '87', '1994')","['1-time Olympic Gold Medal', '1-time World Ch...",Father: Milan Hejduk\r\nCousin: Tomas Hejduk\r...
3,4.0,Alex Tanguay,19,45,1999-00,16,16,1088.0,16.0,82.0,...,11/21/1979,Canada,185,88,L,['Playmaker'],NaN,"('1', '12', '1998')","['1-time CHL All-Rookie Team', '1-time NHL Sta...",Brother: Maxime Tanguay\r\nCousin: Tyler Tangu...
4,5.0,Chris Drury,21,49,1998-99,12,12,892.0,24.0,82.0,...,8/20/1976,USA,178,86,R,NaN,NaN,"('3', '72', '1994')","['2-time Olympic Silver Medal', '1-time World ...",Son: Luke Drury\r\nBrothers: Ted Drury & Jim D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,3889.0,Matt Jones,21,42,2005-06,3,3,106.0,16.0,45.0,...,8/8/1983,USA,183,98,L,NaN,NaN,"('3', '80', '2002')",['1-time USHL Clark Cup Champion'],Brother: Zach Jones
2372,3896.0,Boris Valabik,20,39,2007-08,3,3,80.0,7.0,50.0,...,2/14/1986,Slovakia,201,105,L,"['Defensive Defenseman', 'Enforcer', 'Heavy Hi...",NaN,"('1', '10', '2004')","['1-time U18 WJC Silver Medal', '1-time AHL Ca...",A defenseman with a physical style. Useful in ...
2373,3897.0,Viktor Tikhonov,19,37,2008-09,2,2,111.0,11.0,61.0,...,5/12/1988,Russia,187,87,R,NaN,NaN,"('1', '28', '2008')","['1-time World Championship Gold Medal', '1-ti...",Tikhonov is a forward with fine offensive abil...
2374,3910.0,Mario Kempe,28,37,2017-18,2,2,70.0,18.0,52.0,...,9/19/1988,Sweden,183,86,L,NaN,NaN,"('5', '122', '2007')",['1-time U20 WJC Silver Medal'],Kempe is very good skater with good accelerati...


In [36]:
# Divide players into forwards and defensemen
nhl_defensemen_id = nhl_players_without_rookies_and_fringe[
    nhl_players_without_rookies_and_fringe['player_pos_official'] == 'D'
]['player_id']

In [37]:
nhl_defensemen_id

5          6.0
8          9.0
10        11.0
12        13.0
13        14.0
         ...  
2368    3876.0
2369    3878.0
2371    3889.0
2372    3896.0
2375    3913.0
Name: player_id, Length: 806, dtype: float64

In [38]:
# Get Defensemen
nhl_defensemen = nhl_players_without_rookies_and_fringe[
    nhl_players_without_rookies_and_fringe['player_id'].isin(nhl_defensemen_id)
]

In [39]:
nhl_defensemen

,player_id,player_name_official,rookie_age,age_by_2025,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,...,date_of_birth,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
5,6.0,Ray Bourque,18,64,1979-80,22,22,1612.0,14.0,82.0,...,12/28/1960,Canada,180,98,L,"['Heavy Shooter', 'Offensive Defenseman']",NaN,"('1', '8', '1979')","['2-time Canada Cup Champion', '1-time Hockey ...",Brother: Richard Bourque\r\nSons: Chris Bourqu...
8,9.0,Martin Skoula,19,45,1999-00,10,10,776.0,17.0,82.0,...,10/28/1979,Czechia,190,101,L,NaN,NaN,"('1', '17', '1998')","['1-time World Championship Silver Medal', '1-...",Škoula was a big defenseman with all-around ab...
10,11.0,Greg De Vries,22,52,1995-96,13,13,878.0,6.0,82.0,...,1/4/1973,Canada,190,95,L,NaN,NaN,NaN,['1-time NHL Stanley Cup Champion'],Son: Luke De Vries\r\nNephew: Nate Tivey
12,13.0,Jon Klemm,20,55,1991-92,15,13,773.0,4.0,82.0,...,1/8/1970,Canada,188,92,R,NaN,NaN,NaN,"['1-time CHL Memorial Cup Champion', '2-time N...",NaN
13,14.0,Adam Foote,19,54,1991-92,19,19,1154.0,12.0,81.0,...,7/10/1971,Canada,188,103,R,['Defensive Defenseman'],NaN,"('2', '22', '1989')","['1-time Olympic Gold Medal', '1-time World Cu...","Sons: Cal Foote, Nolan Foote"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368,3876.0,Adam Burt,18,56,1988-89,13,12,737.0,5.0,78.0,...,1/15/1969,USA,188,93,L,NaN,NaN,"('2', '39', '1987')",['1-time OHL Second All-Star Team'],NaN
2369,3878.0,David Harlock,21,54,1993-94,8,4,212.0,1.0,70.0,...,3/16/1971,Canada,188,88,L,NaN,NaN,"('2', '24', '1990')",['1-time Olympic Silver Medal'],NaN
2371,3889.0,Matt Jones,21,42,2005-06,3,3,106.0,16.0,45.0,...,8/8/1983,USA,183,98,L,NaN,NaN,"('3', '80', '2002')",['1-time USHL Clark Cup Champion'],Brother: Zach Jones
2372,3896.0,Boris Valabik,20,39,2007-08,3,3,80.0,7.0,50.0,...,2/14/1986,Slovakia,201,105,L,"['Defensive Defenseman', 'Enforcer', 'Heavy Hi...",NaN,"('1', '10', '2004')","['1-time U18 WJC Silver Medal', '1-time AHL Ca...",A defenseman with a physical style. Useful in ...


In [40]:
# Forwards are all others
nhl_forwards_id = nhl_players_without_rookies_and_fringe[
    ~nhl_players_without_rookies_and_fringe['player_id'].isin(nhl_defensemen_id)
]['player_id']

In [41]:
nhl_forwards_id

0          1.0
1          2.0
2          3.0
3          4.0
4          5.0
         ...  
2364    3869.0
2365    3870.0
2370    3880.0
2373    3897.0
2374    3910.0
Name: player_id, Length: 1570, dtype: float64

In [42]:
# Get Forwards
nhl_forwards = nhl_players_without_rookies_and_fringe[
    nhl_players_without_rookies_and_fringe['player_id'].isin(nhl_forwards_id)
]

In [43]:
nhl_forwards

,player_id,player_name_official,rookie_age,age_by_2025,first_season,seasons_played,seasons_with_gp_over_6,gp_regular_total,gp_regular_min_per_season,gp_regular_max_per_season,...,date_of_birth,nation,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,1.0,Joe Sakic,18,56,1988-89,20,20,1378.0,15.0,84.0,...,7/7/1969,Canada,180,88,L,NaN,NaN,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...
1,2.0,Peter Forsberg,20,52,1994-95,13,12,708.0,2.0,82.0,...,7/20/1973,Sweden,185,93,L,"['Cerebral Tactician', 'Playmaker', 'Two-Way C...",NaN,"('1', '6', '1991')","['1-time TV-Pucken Gold Medal', '1-time J18 SM...",Arguably the greatest Swedish forward to ever ...
2,3.0,Milan Hejduk,21,49,1998-99,14,14,1020.0,29.0,82.0,...,2/14/1976,Czechia / USA,183,87,R,['Sniper'],NaN,"('4', '87', '1994')","['1-time Olympic Gold Medal', '1-time World Ch...",Father: Milan Hejduk\r\nCousin: Tomas Hejduk\r...
3,4.0,Alex Tanguay,19,45,1999-00,16,16,1088.0,16.0,82.0,...,11/21/1979,Canada,185,88,L,['Playmaker'],NaN,"('1', '12', '1998')","['1-time CHL All-Rookie Team', '1-time NHL Sta...",Brother: Maxime Tanguay\r\nCousin: Tyler Tangu...
4,5.0,Chris Drury,21,49,1998-99,12,12,892.0,24.0,82.0,...,8/20/1976,USA,178,86,R,NaN,NaN,"('3', '72', '1994')","['2-time Olympic Silver Medal', '1-time World ...",Son: Luke Drury\r\nBrothers: Ted Drury & Jim D...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,3869.0,Dean Sylvester,25,52,1998-99,3,2,96.0,1.0,52.0,...,12/30/1972,USA,188,91,R,NaN,NaN,NaN,[],"Brothers: Evan Sylvester, Todd Sylvester"
2365,3870.0,Mike Sullivan,22,57,1991-92,11,11,709.0,19.0,81.0,...,2/27/1968,USA,188,84,L,NaN,NaN,"('4', '69', '1987')",[],Son: Matt Sullivan\r\nSon-in-law: Charlie McAvoy
2370,3880.0,Jean-Pierre Vigier,23,49,2000-01,6,5,213.0,2.0,72.0,...,9/11/1976,Canada,182,86,R,NaN,NaN,NaN,"['1-time AHL Calder Cup Champion', '1-time AHL...","Cousins: Ian Vigier, Stephan Vigier"
2373,3897.0,Viktor Tikhonov,19,37,2008-09,2,2,111.0,11.0,61.0,...,5/12/1988,Russia,187,87,R,NaN,NaN,"('1', '28', '2008')","['1-time World Championship Gold Medal', '1-ti...",Tikhonov is a forward with fine offensive abil...


## Write to CSV

In [44]:
# Write nhl_defensemen to CSV
nhl_defensemen.to_csv('../dataset/preprocess/nhl_defensemen.csv', index=False, encoding='utf-8-sig')

In [45]:
# Write nhl_forwards to CSV
nhl_forwards.to_csv('../dataset/preprocess/nhl_forwards.csv', index=False, encoding='utf-8-sig')

## Get the First 3 Seasons' Career Stats

- Get the first 3 seasons' career stats for players not in the rookies and fringe player set.

- This will be used to train the Logistic Regression model to predict rookies' tier.

In [52]:
# Load Main Players
nhl_main_players = pd.read_csv('../dataset/preprocess/nhl_players_career_stats_with_metadata_without_rookies_and_fringe.csv')
nhl_main_players.shape

(2376, 71)

In [53]:
# Get the ID of these main players
main_players_id = nhl_main_players['player_id']

# Load the NHL Players Stats
nhl_players_stats = pd.read_csv('../dataset/nhl_players_official_stats_with_id_sorted.csv')

# Copy only the main players' stats
nhl_main_players_stats = nhl_players_stats[nhl_players_stats['player_id'].isin(main_players_id)].copy()

# Copy only the NHL stats
nhl_main_players_stats = nhl_main_players_stats[nhl_main_players_stats['league'] == 'NHL'].copy()

nhl_main_players_stats.shape

(24888, 37)

In [54]:
# Get the first 3 seasons' stats for each player
nhl_main_players_3_season_stats = nhl_main_players_stats.groupby('player_id').head(3)
nhl_main_players_3_season_stats

,player_id,player_name_official,season,league,team,gp_regular,g_regular,a_regular,p_regular,plus_minus_regular,...,ppg_playoffs,ppp_playoffs,shg_playoffs,shp_playoffs,toi_per_game_playoffs,gwg_playoffs,otg_playoffs,sog_playoffs,shooting_pct_playoffs,fo_pct_playoffs
6,1.0,Joe Sakic,1988-89,NHL,Quebec Nordiques,70.0,23.0,39.0,62.0,-36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,Joe Sakic,1989-90,NHL,Quebec Nordiques,80.0,39.0,63.0,102.0,-40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,Joe Sakic,1990-91,NHL,Quebec Nordiques,80.0,48.0,61.0,109.0,-26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2.0,Peter Forsberg,1994-95,NHL,Quebec Nordiques,47.0,15.0,35.0,50.0,17.0,...,1.0,1.0,0.0,0.0,NaN,0.0,0.0,13.0,15.4,NaN
51,2.0,Peter Forsberg,1995-96,NHL,Colorado Avalanche,82.0,30.0,86.0,116.0,26.0,...,3.0,6.0,0.0,0.0,NaN,1.0,0.0,50.0,20.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105630,3897.0,Viktor Tikhonov,2015-16,NHL,Arizona Coyotes,39.0,3.0,3.0,6.0,-6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105968,3910.0,Mario Kempe,2017-18,NHL,Arizona Coyotes,18.0,2.0,2.0,4.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105970,3910.0,Mario Kempe,2018-19,NHL,Arizona Coyotes,52.0,4.0,5.0,9.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106052,3913.0,Dysin Mayo,2021-22,NHL,Arizona Coyotes,67.0,4.0,8.0,12.0,-22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Write to CSV
nhl_main_players_3_season_stats.to_csv('../dataset/preprocess/nhl_main_players_3_season_stats.csv', index=False, encoding='utf-8-sig')

## Aggregate First 3 Seasons' Stats

In [93]:
# Aggregate first 3 seasons' career stats
# Load first 3 seasons' stats
first_3_seasons_stats = pd.read_csv('../dataset/preprocess/nhl_main_players_3_season_stats.csv')

# Step 1: Copy the first 3 seasons' stats
first_3_seasons_stats = nhl_main_players_3_season_stats.copy()

first_3_seasons_stats

,player_id,player_name_official,season,league,team,gp_regular,g_regular,a_regular,p_regular,plus_minus_regular,...,ppg_playoffs,ppp_playoffs,shg_playoffs,shp_playoffs,toi_per_game_playoffs,gwg_playoffs,otg_playoffs,sog_playoffs,shooting_pct_playoffs,fo_pct_playoffs
6,1.0,Joe Sakic,1988-89,NHL,Quebec Nordiques,70.0,23.0,39.0,62.0,-36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,Joe Sakic,1989-90,NHL,Quebec Nordiques,80.0,39.0,63.0,102.0,-40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,Joe Sakic,1990-91,NHL,Quebec Nordiques,80.0,48.0,61.0,109.0,-26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2.0,Peter Forsberg,1994-95,NHL,Quebec Nordiques,47.0,15.0,35.0,50.0,17.0,...,1.0,1.0,0.0,0.0,NaN,0.0,0.0,13.0,15.4,NaN
51,2.0,Peter Forsberg,1995-96,NHL,Colorado Avalanche,82.0,30.0,86.0,116.0,26.0,...,3.0,6.0,0.0,0.0,NaN,1.0,0.0,50.0,20.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105630,3897.0,Viktor Tikhonov,2015-16,NHL,Arizona Coyotes,39.0,3.0,3.0,6.0,-6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105968,3910.0,Mario Kempe,2017-18,NHL,Arizona Coyotes,18.0,2.0,2.0,4.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105970,3910.0,Mario Kempe,2018-19,NHL,Arizona Coyotes,52.0,4.0,5.0,9.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106052,3913.0,Dysin Mayo,2021-22,NHL,Arizona Coyotes,67.0,4.0,8.0,12.0,-22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# Convert columns that should be numeric
numeric_cols = [
    "gp_regular", "g_regular", "a_regular", "p_regular",
    "plus_minus_regular", "pim_regular", "ppg_regular", "ppp_regular",
    "shg_regular", "shp_regular", "gwg_regular", "otg_regular", "sog_regular",
    "shooting_pct_regular", "fo_pct_regular",
    "gp_playoffs", "g_playoffs", "a_playoffs", "p_playoffs",
    "plus_minus_playoffs", "pim_playoffs", "ppg_playoffs", "ppp_playoffs",
    "shg_playoffs", "shp_playoffs", "gwg_playoffs", "otg_playoffs", "sog_playoffs",
    "shooting_pct_playoffs", "fo_pct_playoffs"
]

for col in numeric_cols:
    nhl_career_stats[col] = pd.to_numeric(nhl_career_stats[col], errors="coerce")

# Handle TOI (time on ice) separately if it's "MM:SS" or "HH:MM:SS"
def convert_toi_to_minutes(toi_str):
    if pd.isna(toi_str): return None
    parts = str(toi_str).split(":")
    if len(parts) == 2:  # MM:SS
        m, s = map(int, parts)
        return m + s/60
    elif len(parts) == 3:  # HH:MM:SS
        h, m, s = map(int, parts)
        return h*60 + m + s/60
    return None

first_3_seasons_stats["toi_per_game_regular"] = first_3_seasons_stats["toi_per_game_regular"].apply(convert_toi_to_minutes)
first_3_seasons_stats["toi_per_game_playoffs"] = first_3_seasons_stats["toi_per_game_playoffs"].apply(convert_toi_to_minutes)

# Fill NaN with 0
first_3_seasons_stats = first_3_seasons_stats.fillna(0)

# Add extra stats before aggregation
agg_funcs = {
    "gp_regular": ["sum", "min", "max"],  # total GP, min GP, max GP in a season
    "g_regular": "sum",
    "a_regular": "sum",
    "p_regular": "sum",
    "plus_minus_regular": "sum",
    "pim_regular": "sum",
    "ppg_regular": "sum",
    "ppp_regular": "sum",
    "shg_regular": "sum",
    "shp_regular": "sum",
    "gwg_regular": "sum",
    "otg_regular": "sum",
    "sog_regular": "sum",
    "shooting_pct_regular": "mean",  # percentage -> average
    "fo_pct_regular": "mean",        # percentage -> average
    "toi_per_game_regular": "mean",  # average TOI per game
    "gp_playoffs": "sum",
    "g_playoffs": "sum",
    "a_playoffs": "sum",
    "p_playoffs": "sum",
    "plus_minus_playoffs": "sum",
    "pim_playoffs": "sum",
    "ppg_playoffs": "sum",
    "ppp_playoffs": "sum",
    "shg_playoffs": "sum",
    "shp_playoffs": "sum",
    "gwg_playoffs": "sum",
    "otg_playoffs": "sum",
    "sog_playoffs": "sum",
    "shooting_pct_playoffs": "mean",  # %
    "fo_pct_playoffs": "mean",        # %
    "toi_per_game_playoffs": "mean",
    "season": "nunique"  # number of distinct NHL seasons
}

# Aggregate
first_3_seasons_stats = first_3_seasons_stats.groupby(
    ["player_id", "player_name_official"]
).agg(agg_funcs)


In [95]:
first_3_seasons_stats

gp_regular             g_regular a_regular  \
                                      sum   min   max       sum       sum   
player_id player_name_official                                              
1.0       Joe Sakic                 230.0  70.0  80.0     110.0     163.0   
2.0       Peter Forsberg            194.0  47.0  82.0      73.0     179.0   
3.0       Milan Hejduk              244.0  80.0  82.0      91.0     108.0   
4.0       Alex Tanguay              228.0  70.0  82.0      57.0     119.0   
5.0       Chris Drury               232.0  71.0  82.0      64.0     112.0   
...                                   ...   ...   ...       ...       ...   
3889.0    Matt Jones                106.0  16.0  45.0       1.0      10.0   
3896.0    Boris Valabik              80.0   7.0  50.0       0.0       7.0   
3897.0    Viktor Tikhonov           111.0  11.0  61.0      11.0      11.0   
3910.0    Mario Kempe                70.0  18.0  52.0       6.0       7.0   
3913.0    Dysin Mayo                 82.0  15.0  67.0       4.0       8.0   

                               p_regular plus_minus_regular pim_regular  \
                                     sum                sum         sum   
player_id player_name_official                                            
1.0       Joe Sakic                273.0             -102.0        75.0   
2.0       Peter Forsberg           252.0               74.0       136.0   
3.0       Milan Hejduk             199.0               54.0        78.0   
4.0       Alex Tanguay             176.0               49.0        95.0   
5.0       Chris Drury              176.0               23.0       151.0   
...                                  ...                ...         ...   
3889.0    Matt Jones                11.0              -27.0        63.0   
3896.0    Boris Valabik              7.0              -14.0       210.0   
3897.0    Viktor Tikhonov           22.0              -13.0        40.0   
3910.0    Mario Kempe               13.0                4.0        22.0   
3913.0    Dysin Mayo                12.0              -30.0        35.0   

                               ppg_regular ppp_regular  ... ppp_playoffs  \
                                       sum         sum  ...          sum   
player_id player_name_official                          ...                
1.0       Joe Sakic                   30.0       100.0  ...          0.0   
2.0       Peter Forsberg              15.0        80.0  ...         15.0   
3.0       Milan Hejduk                29.0        65.0  ...         17.0   
4.0       Alex Tanguay                19.0        47.0  ...         12.0   
5.0       Chris Drury                 24.0        66.0  ...          7.0   
...                                    ...         ...  ...          ...   
3889.0    Matt Jones                   0.0         0.0  ...          0.0   
3896.0    Boris Valabik                0.0         0.0  ...          0.0   
3897.0    Viktor Tikhonov              3.0         3.0  ...          0.0   
3910.0    Mario Kempe                  0.0         0.0  ...          0.0   
3913.0    Dysin Mayo                   0.0         0.0  ...          0.0   

                               shg_playoffs shp_playoffs gwg_playoffs  \
                                        sum          sum          sum   
player_id player_name_official                                          
1.0       Joe Sakic                     0.0          0.0          0.0   
2.0       Peter Forsberg                0.0          0.0          1.0   
3.0       Milan Hejduk                  0.0          0.0          5.0   
4.0       Alex Tanguay                  0.0          0.0          3.0   
5.0       Chris Drury                   0.0          1.0          8.0   
...                                     ...          ...          ...   
3889.0    Matt Jones                    0.0          0.0          0.0   
3896.0    Boris Valabik                 0.0          0.0          0.0   
3897.0    Viktor Tikhonov        

## Write to CSV

In [96]:
first_3_seasons_stats.to_csv('../dataset/preprocess/nhl_main_players_3_season_stats_aggregated.csv', index=False, encoding='utf-8-sig')
